In [1]:
import json
import json_coder
import pandas as pd
import networkx as nx
import befaas as bf
import befaas.logentry as le
from befaas.graph import build_function_graph, add_default_metadata, conv_to_ms

filepath = "../dumps/"
dump = "uniqueIoT.json"
outfile = "../img/latenciesIoT.pkl"

print("Everything is set up.")

Everything is set up.


In [2]:
print("Load dump... (may take a while)")
data = bf.load_logs(filepath + dump)
print("Done.")

Load dump... (may take a while)
Done.


In [3]:
print("Sort data to contexts...")
contexts = {}

for entry in data:
    id = entry.context_id

    if id == None:
        continue

    if not (id in contexts):
        contexts[id] = []

    contexts[id].append(entry)
print("Done.")

Sort data to contexts...
Done.


In [4]:
print("Find xpairs...")
xpairs = {}

for ctx_id in contexts:
    ctx = contexts[ctx_id]
    for entry in ctx:
        id = entry.event["xPair"]
        if not (id in xpairs):
            xpairs[id] = []

        specialPair = False
        if isinstance(entry, le.PerfLog):
            if not entry.perf_type_data == "":
                parts = entry.perf_type_data.split(":")
                if len(parts) > 1:
                    id = parts[1]
                    if not (id in xpairs):
                        xpairs[id] = []
                    xpairs[id].append(entry)
                    specialPair = True

        if not specialPair:
            xpairs[id].append(entry)
print("done.")

Find xpairs...
done.


In [5]:
print("Calculate transport times...")
plotdata = []
debugpairs = []
debugpairs2 = []

for xpairID in xpairs:
    xpair = xpairs[xpairID]

    src = ""
    dst = ""
    platform = ""
    latency = 0
    start = None

    type = ""
    hasDB = False
    for entry in xpair:
        if isinstance(entry, le.ArtilleryLog):
            type = "Load"
            break

    if type == "":
        for entry in xpair:
            if isinstance(entry, le.PerfLog):
                if entry.perf_type[1] == "dbOut":
                    hasDB = True
                    break

    if type == "":
        rpcInFound = False
        rpcOutFound = False

        for entry in xpair:
            if isinstance(entry, le.PerfLog):
                if entry.type == "measure":
                    if entry.perf_type[1] == "rpcIn":
                        rpcInFound = True
                    if entry.perf_type[1] == "rpcOut":
                        rpcOutFound = True

        if rpcInFound and rpcOutFound:
            type = "Call"

    if hasDB:
        dst = "DB"
        for entry in xpair:
            if isinstance(entry, le.PerfLog):
                if entry.type == "measure":
                    if entry.perf_type[1] == "dbOut":
                        latency += entry.perf["duration"]
                        src = entry.function

        if "lightphase" in src:
            src = "Light Phase"
            platform = "Google"
        elif "trafficstat" in src:
            src = "Traffic Statistic"
            platform = "Google"
        elif "movementplan" in src:
            src = "Movement Plan"
            platform = "Both"

        #print(f"DB: {src} - {dst} | {latency}")
        #row = {}
        #row["pair"] = src + " - " + dst
        #row["latency"] = latency
        #row["platform"] = platform
        #plotdata.append(row)

    src = ""
    dst = ""
    latency = 0
    platform = ""

    if type == "Load":

        src = "Artillery"
        platform = "AWS → tinyFaaS"
        artilleryStart = None
        artilleryEnd = None
        duration = 0

        for entry in xpair:
            if isinstance(entry, le.ArtilleryLog):
                if (entry.event["type"] == "before"):
                    artilleryStart = entry.timestamp

                if (entry.event["type"] == "after"):
                    artilleryEnd = entry.timestamp

            if isinstance(entry, le.PerfLog):
                if entry.type == "measure":
                    if entry.perf_type[1] == "post":
                        duration = entry.perf["duration"]
                        dst = entry.function
                    if entry.perf_type[1] == "rpcIn":
                        duration = entry.perf["duration"]
                        dst = entry.function

                    if "weathersensor" in dst:
                        platform = "AWS → GCP"

        latency = (artilleryEnd.timestamp() - artilleryStart.timestamp()) * 1000 # to ms
        latency -= duration

        if "trafficsensor" in dst:
            dst = "Traffic Sensor"
        elif "weathersensor" in dst:
            dst = "Weather Sensor"
        elif "objectrec" in dst:
            dst = "Object Recognition"

        start = artilleryStart

        #print(f"Load: {src} - {dst} | {platform} | {latency}")

    if type == "Call":
        srcDuration = 0
        dstDuration = 0
        for entry in xpair:
            if isinstance(entry, le.PerfLog):
                if entry.type == "measure":
                    if entry.perf_type[1] == "rpcIn":
                        dstDuration = entry.perf["duration"]
                        dst = entry.function

                    if entry.perf_type[1] == "rpcOut":
                        srcDuration += entry.perf["duration"]
                        src = entry.function
                if entry.type == "mark":
                    if entry.perf_type[1] == "rpcOut":
                        start = entry.timestamp

        latency = srcDuration - dstDuration

        if latency > 10000:
            debugpairs2.append(xpair)
        #print(f"Call: {src} - {dst} | {latency}")

    if type == "":
        debugpairs.append(xpair)

    if "roadcondition" in src:
        platform = "Google"
        src = "Road Condition"
        dst = "Light Phase"
    elif "weathersensor" in src:
        platform = "Google"
        src = "Weather Sensor"
        dst = "Road Condition"
    elif "trafficsensor" in src:
        platform = "tinyFaaS"
        src = "Traffic Sensor"
        dst = "Movement Plan"
    elif "movementplan" in src:
        platform = "both"
        src = "Movement Plan"
        dst = "Light Phase"
    elif "emergency" in src:
        platform = "both"
        src = "Emergency"
        dst = "Light Phase"
    elif "objectrec" in src:
        platform = "tinyFaaS"
        src = "Object Recognition"
        if "trafficstat" in dst:
            platform = "both"
            dst = "Traffic Statistic"
        elif "emergency" in dst:
            dst = "Emergency"
        elif "movement" in dst:
            dst = "Movement Plan"

    if src == "":
        print(f"No SRC: {src} - {dst} - {type} - {latency}")
    else:
        if not dst == "DB":
            row = {}
            row["pair"] = src + " - " + dst
            row["latency"] = latency
            row["platform"] = platform
            row["start"] = start
            plotdata.append(row)



Calculate transport times...


In [6]:
df_calls = pd.DataFrame(plotdata)
df_calls.head()
df_calls.describe()

,latency
count,3587.000000
mean,-71.249852
std,481.904120
min,-3112.022038
25%,14.272021
50%,40.313254
75%,86.550390
max,1213.300904


In [7]:
df_calls.to_pickle(outfile)